Задание
В рамках этого итогового задания мы будем прогнозировать сердечную недостаточность.

С описанием датасета можно познакомиться здесь, но сам датасет для обучения нужно будет скачать тут, датасет для тестирования — тут.

Плана по выполнению задания не будет. 
Но есть несколько требований: 

оберните весь конвейер преобразований в Pipeline

подберите оптимальный вариант прогнозной модели с помощью GridSearchCV

примените обученный на тренировочных данных конвейер к тестовым данным, никак не предобрабатывая их, а лишь загрузив из файла и отделив целевой признак от остальных

получите на тестовой части качество не ниже 0.87 по метрике ROCAUC

In [87]:
import pandas as pd
from sklearn.model_selection import  GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.impute import SimpleImputer

In [88]:
data_train = pd.read_csv('heart_adapt_train.csv')
data_train.head(10).T

,0,1,2,3,4,5,6,7,8,9
Age,74.0,58.0,44.0,50.0,NaN,51.0,53.0,38.0,56.0,61.0
Sex,M,M,M,M,M,M,M,M,M,M
ChestPainType,NAP,NAP,ATA,ASY,ASY,NAP,ASY,NAP,NAP,ASY
RestingBP,138.0,132.0,150.0,144.0,145.0,135.0,154.0,138.0,125.0,190.0
Cholesterol,NaN,224.0,288.0,349.0,248.0,160.0,NaN,175.0,NaN,287.0
FastingBS,0,0,0,0,0,0,1,0,1,1
RestingECG,Normal,LVH,Normal,LVH,Normal,Normal,ST,Normal,Normal,LVH
MaxHR,116,173,150,120,96,150,140,173,98,150
ExerciseAngina,N,N,Y,Y,Y,N,Y,N,N,Y
Oldpeak,0.2,3.2,3.0,1.0,2.0,2.0,1.5,0.0,-2.0,2.0


In [89]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             533 non-null    float64
 1   Sex             589 non-null    object 
 2   ChestPainType   589 non-null    object 
 3   RestingBP       588 non-null    float64
 4   Cholesterol     462 non-null    float64
 5   FastingBS       589 non-null    int64  
 6   RestingECG      589 non-null    object 
 7   MaxHR           589 non-null    int64  
 8   ExerciseAngina  589 non-null    object 
 9   Oldpeak         589 non-null    float64
 10  ST_Slope        589 non-null    object 
 11  HeartDisease    589 non-null    int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 55.3+ KB


In [90]:
data_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,533.0,54.195122,9.532661,28.0,48.0,55.0,61.00,77.0
RestingBP,588.0,133.358844,18.851852,80.0,120.0,130.0,144.00,200.0
Cholesterol,462.0,245.632035,58.599184,85.0,209.0,240.0,279.75,603.0
FastingBS,589.0,0.258065,0.437942,0.0,0.0,0.0,1.00,1.0
MaxHR,589.0,134.893039,24.942596,63.0,117.0,135.0,154.00,195.0
Oldpeak,589.0,0.937521,1.071318,-2.6,0.0,0.8,1.60,5.0
HeartDisease,589.0,0.646859,0.478352,0.0,0.0,1.0,1.00,1.0


Пропуски есть в колонках 'Age', 'RestingBP' и 'Cholesterol'. Их можно будет заменить на средние значения

In [91]:
# отделяем признаки от целевой переменной
features_train = data_train.drop(['HeartDisease'], axis=1)
target_train =data_train['HeartDisease']

In [92]:
target_train.value_counts()

HeartDisease
1    381
0    208
Name: count, dtype: int64

Целевая переменная не сбалансирована

In [93]:
# класс для приведения категориальных переменных к числовому типу
class CategoriesToFlags(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.get_dummies(X, drop_first=True)

In [94]:
# загружаем тестовые данные
data_test = pd.read_csv('heart_adapt_test.csv')
data_test.head().T

,0,1,2,3,4
Age,44.0,63.0,35.0,69.0,NaN
Sex,M,M,F,M,M
ChestPainType,NAP,ASY,TA,NAP,TA
RestingBP,130.0,130.0,120.0,140.0,142.0
Cholesterol,233.0,308.0,160.0,NaN,200.0
FastingBS,0,0,0,1,1
RestingECG,Normal,Normal,ST,ST,ST
MaxHR,179,138,185,118,100
ExerciseAngina,Y,Y,N,N,N
Oldpeak,0.4,2.0,0.0,2.5,1.5


In [95]:
# отделяем признаки от целевой переменной
features_test = data_test.drop(['HeartDisease'], axis=1)
target_test = data_test['HeartDisease']

In [96]:
# создаем Pipeline
pipe = Pipeline(steps=[
    ('ohe_types', CategoriesToFlags()), # приведение категориальных признаков к числовому типу
    ('imputer', SimpleImputer(strategy='mean')),  # заполнение пропусков
    ('scaler', StandardScaler()),  # нормализация признаков
    ('classify', DecisionTreeClassifier(class_weight='balanced', random_state=0))  # создание модели
])


In [97]:
pipe.fit(X=features_train, y=target_train)
print('Качество модели на обучающей выборке:', {roc_auc_score(target_train, pipe.predict_proba(features_train)[:, 1])})

print('Качество модели на тестовой выборке:', {roc_auc_score(target_test, pipe.predict_proba(features_test)[:, 1])})

Качество модели на обучающей выборке: {1.0}
Качество модели на тестовой выборке: {0.8305961754780653}


Модель переобучилась

In [98]:
params = [
    {'classify': [LogisticRegression(class_weight='balanced', random_state=0)]}, 
    {'classify': [DecisionTreeClassifier(class_weight='balanced', random_state=0)], 'classify__max_depth': [2, 5, 10, 20]}
]

grid_search = GridSearchCV(pipe, param_grid=params, cv=5, scoring='roc_auc')
grid_search.fit(X=features_train, y=target_train)

print('Качество модели на тестовой выборке c лучшей моделью:', 
    {roc_auc_score(target_test, grid_search.predict_proba(features_test)[:, 1])})
print('Параметры лучшей модели:', grid_search.best_params_)

Качество модели на тестовой выборке c лучшей моделью: {0.9044994375703037}
Параметры лучшей модели: {'classify': LogisticRegression(class_weight='balanced', random_state=0)}


### Вывод:

При создании модели прогнозирования сердечной недостаточности удалось достичь качество на тестовой выборке 0.9044994375703037.
Лучшей моделью оказалась LogisticRegression .